In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
pd.options.display.max_columns
df = pd.read_csv('df_all_rs.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('index', axis=1)
df = df.sort_values(by="GAME_DATE_EST")
df

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
1188,2003-10-28,20300003,1610612747,1610612742,2003,1610612747,109.0,0.506,0.600,0.350,32.0,46.0,1610612742,93.0,0.376,0.733,0.280,17.0,46.0,1
1187,2003-10-28,20300002,1610612759,1610612756,2003,1610612759,83.0,0.425,0.769,0.100,20.0,38.0,1610612756,82.0,0.361,0.810,0.500,14.0,43.0,1
1186,2003-10-28,20300001,1610612755,1610612748,2003,1610612755,89.0,0.440,0.533,0.350,25.0,39.0,1610612748,74.0,0.408,0.824,0.250,16.0,44.0,1
1185,2003-10-29,20300012,1610612762,1610612757,2003,1610612762,99.0,0.575,0.714,0.556,25.0,29.0,1610612757,92.0,0.429,0.762,0.333,20.0,40.0,1
1184,2003-10-29,20300006,1610612740,1610612737,2003,1610612740,88.0,0.324,0.700,0.160,24.0,55.0,1610612737,83.0,0.398,0.737,0.214,18.0,58.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18173,2019-04-10,21801225,1610612763,1610612744,2018,1610612763,132.0,0.490,0.833,0.457,25.0,49.0,1610612744,117.0,0.500,0.857,0.433,32.0,39.0,1
18172,2019-04-10,21801224,1610612755,1610612741,2018,1610612755,125.0,0.559,0.600,0.462,30.0,47.0,1610612741,109.0,0.474,0.667,0.375,21.0,39.0,1
18171,2019-04-10,21801223,1610612752,1610612765,2018,1610612752,89.0,0.403,0.720,0.300,23.0,33.0,1610612765,115.0,0.482,0.950,0.424,25.0,56.0,0
18170,2019-04-10,21801222,1610612766,1610612753,2018,1610612766,114.0,0.526,0.889,0.276,21.0,32.0,1610612753,122.0,0.545,0.750,0.367,24.0,39.0,0


In [3]:
x = df[['FG_PCT_home','FG_PCT_away','FG3_PCT_home','FG3_PCT_away','AST_home','REB_home','AST_away','REB_away']]
y = df['HOME_TEAM_WINS']

X = x.copy()
X_normalized = (X - X.mean()) / X.std()

X_train, X_valid, y_train, y_valid = train_test_split(X_normalized,y,train_size=0.70,test_size=0.30,random_state=0)

In [4]:
columnas = ['Tecnica','whatever']

df_opt = pd.DataFrame(columns=columnas)
df_opt.set_index("Tecnica",inplace=True)
df_opt.loc['Standard'] = [0]
df_opt.loc['GridSearch'] = [0]
df_opt.loc['RandomSearch'] = [0]
df_opt.loc['Hyperopt'] = [0]
df_opt.head()

,whatever
Tecnica,
Standard,0
GridSearch,0
RandomSearch,0
Hyperopt,0


In [25]:
knr = KNeighborsRegressor()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
mlpr = MLPRegressor()
logR = LogisticRegression()
svr = SVR()

#[knr,dtr,rfr,mlpr,logR,svr]
models = [svr]

col = 0
for model in models:
    model.fit(X_train,y_train.values.ravel())
    df_opt.iloc[0,col] = model.score(X_valid,y_valid)
    col += 1

df_opt.head()

,whatever
Tecnica,
Standard,0.498843
GridSearch,0.838316
RandomSearch,0
Hyperopt,0


In [5]:
# KNeighborsRegressor
n_neighbors = range(1, 5, 2)
weights = ['uniform', 'distance']
#algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
#leaf_size = range(1,41,2)
#metric = ['euclidean', 'manhattan', 'minkowski']

knr_grid = dict(n_neighbors=n_neighbors, 
                weights=weights)


# DecisionTreeRegressor
#criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
max_depth = range(1, 4, 2)
min_samples_leaf = range(1, 4, 2)

dtr_grid = dict(max_depth=max_depth,
                min_samples_leaf=min_samples_leaf)


# RandomForestRegressor
n_estimators = [10, 100, 1000]
max_depth = range(1, 5, 2)
min_samples_leaf = range(1, 5, 2)
max_features = ['sqrt', 'log2']

rfr_grid = dict(n_estimators=n_estimators, 
                max_depth=max_depth, 
                min_samples_leaf=min_samples_leaf,
                max_features=max_features)


# MLPRegressor
activation = ['relu', 'logistic', 'tanh']
solver = ['adam', 'sgd']#, 'lbfgs']
learning_rate = ['constant', 'invscaling', 'adaptive']
#batch_size = range(1, 101, 50)

mlpr_grid = dict(activation=activation,
                 solver=solver,
                 learning_rate=learning_rate)


# LogisticRegression
solvers = ['newton-cg', 'liblinear']#, 'lbfgs']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
#class_weight

logR_grid = dict(solver=solvers,
                 penalty=penalty,
                 C=c_values)


#SupportVectorRegressor
kernels =['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
c_values = [100, 10, 1.0, 0.1, 0.01]

svr_grid = dict(kernel=kernels,
                C=c_values)

In [29]:
#[knr,dtr,rfr,mlpr,logR,svr]
models = [svr]

#[knr_grid, dtr_grid, rfr_grid, mlpr_grid, logR_grid, svr_grid]
grids = [svr_grid]
col = 0

for ind in range(0,len(models)):
    cv = RepeatedStratifiedKFold(n_splits=10, 
                                 n_repeats=3, 
                                 random_state=1)
    
    grid_search = GridSearchCV(estimator=models[col],
                               param_grid=grids[col], 
                               n_jobs=-1, 
                               cv=cv,
                               scoring='accuracy',
                               error_score=0)
    
    grid_clf_acc = grid_search.fit(X_train, y_train)
    df_opt.iloc[1,col] = grid_clf_acc.score(X_valid,y_valid)
    col += 1

df_opt.head()

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

KeyboardInterrupt: 

In [6]:
df_opt.head()

,whatever
Tecnica,
Standard,0
GridSearch,0
RandomSearch,0
Hyperopt,0
